##### 1581. Customer Who Visited but Did Not Make Any Transactions


```sql
SELECT v.customer_id, COUNT(v.visit_id) AS count_no_trans
FROM Visits v
LEFT JOIN Transactions t ON v.visit_id = t.visit_id
WHERE t.transaction_id IS NULL
GROUP BY v.customer_id;
``````

##### 197. Rising Temperature

- 같은 테이블 내에서 전 날보다 기온이 낮은 열들을 추출하는 문제

- Weather a의 복제본 Weather b를 만든다.
- 기존 테이블에 DATE_ADD(b.recordDate, INTERVAL 1 DAY)로 하루씩 밀린 날짜 column을 더해준다.
- 위에서 만든 column을 기준으로 ON하여 두 테이블을 JOIN해준다.
- WHERE로 a.temperature > b.temperature 설정

```sql
SELECT a.id
FROM Weather a
     JOIN Weather b ON a.recordDate = DATE_ADD(b.recordDate, INTERVAL 1 DAY)
WHERE a.temperature > b.temperature;
``````

##### 1661. Average Time of Process per Machine


```sql
SELECT 
    start.machine_id, 
    ROUND(AVG(end.timestamp - start.timestamp), 3) AS processing_time
FROM 
    Activity AS start
JOIN 
    Activity AS end ON start.machine_id = end.machine_id 
                    AND start.process_id = end.process_id 
                    AND start.activity_type = 'start' 
                    AND end.activity_type = 'end'
GROUP BY 
    start.machine_id;
``````

아래는 위처럼 간단하게 할 수 있는 테이블을 하나하나 생성하며 헤멘 버전... 어지간한건 join으로 한번에 해결 가능

```sql
CREATE TABLE s AS
SELECT machine_id, process_id, timestamp
FROM Activity
WHERE activity_type = 'start';

CREATE TABLE e AS
SELECT machine_id, process_id, timestamp
FROM Activity
WHERE activity_type = 'end';

CREATE TABLE ProcessDuration AS
SELECT s.machine_id, 
       s.process_id, 
       e.timestamp - s.timestamp AS duration
FROM s
JOIN e ON s.machine_id = e.machine_id AND s.process_id = e.process_id;

SELECT machine_id, ROUND(AVG(duration),3) processing_time
FROM ProcessDuration
GROUP BY machine_id;
```

위처럼 여러개의 쿼리로 이루어져 있으면 Leetcode에서 오류가난다.\
\
with을 이용하여 하나의 쿼리로 합쳐주면 해결된다.

```sql
WITH s AS (
    SELECT machine_id, process_id, timestamp
    FROM Activity
    WHERE activity_type = 'start'
),
e AS (
    SELECT machine_id, process_id, timestamp
    FROM Activity
    WHERE activity_type = 'end'
),
ProcessDuration AS (
    SELECT s.machine_id, 
           s.process_id, 
           e.timestamp - s.timestamp AS duration
    FROM s
    JOIN e ON s.machine_id = e.machine_id AND s.process_id = e.process_id
)
SELECT machine_id, ROUND(AVG(duration), 3) AS processing_time
FROM ProcessDuration
GROUP BY machine_id;
``````